[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). This notebook is released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).

# DIY Covid-19 Dashboard

This is a template for your DIY Covid Dashboard, to which you can add the code you developed in the previous notebooks. The dashboard will be displayed using [voila](https://voila.readthedocs.io/en/stable/index.html), a Python dashboarding tool that converts notebooks to standalone dashboards. Contrary to the other libraries we have seen, the ```voila``` package must be installed using *pip* or *conda* but it does not need to be imported - it rather acts at the level of the notebook server. Package ```voila``` is already installed on the QMUL JupyterHub as well as in the Binder - to install it locally, follow the [instructions](https://voila.readthedocs.io/en/stable/install.html) online.

Broadly speaking, Voila acts by **running all the cells in your notebook** when the dashboard is first loaded; it then hides all code cells and displays all markdown cells and any outputs, including widgets. However, the code is still there in the background and handles any interaction with the widgets. To view this dashboard template rendered in Voila click [here](https://mybinder.org/v2/gh/fsmeraldi/diy-covid19dash/main?urlpath=%2Fvoila%2Frender%2FDashboard.ipynb).

In [1]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [2]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

## Load initial data from disk

You should include "canned" data in ```.json``` files along with your dashboard. When the dashboard starts, it should load that data and assign it as a dictionary to the ```jsondata``` variable (the code below will be hidden when the dashboard is rendered by Voila).

In [3]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
with open("occipiedBeds.json", "rt") as INFILE:
    jsondata=json.load(INFILE)

{'data': [{'date': '2023-11-24', 'occipiedBeds': None, 'hospital': 1880},
  {'date': '2023-11-17', 'occipiedBeds': None, 'hospital': 2012},
  {'date': '2023-11-10', 'occipiedBeds': None, 'hospital': 2300},
  {'date': '2023-11-03', 'occipiedBeds': None, 'hospital': 2506},
  {'date': '2023-10-27', 'occipiedBeds': None, 'hospital': 3062},
  {'date': '2023-10-20', 'occipiedBeds': None, 'hospital': 3378},
  {'date': '2023-10-13', 'occipiedBeds': None, 'hospital': 3969},
  {'date': '2023-10-06', 'occipiedBeds': None, 'hospital': 4312},
  {'date': '2023-09-29', 'occipiedBeds': None, 'hospital': 3830},
  {'date': '2023-09-22', 'occipiedBeds': None, 'hospital': 3060},
  {'date': '2023-09-15', 'occipiedBeds': None, 'hospital': 2766},
  {'date': '2023-09-08', 'occipiedBeds': None, 'hospital': 3287},
  {'date': '2023-09-01', 'occipiedBeds': None, 'hospital': 2535},
  {'date': '2023-08-25', 'occipiedBeds': None, 'hospital': 2282},
  {'date': '2023-08-18', 'occipiedBeds': None, 'hospital': 2254},
  

## Wrangle the data

The dashboard should contain the logic to wrangle the raw data into a ```DataFrame``` (or more than one, as required) that will be used for plotting. The wrangling code should be put into a function and called on the data from the JSON file (we'll need to call it again on any data downloaded from the API).  In this template, we just pretend we are wrangling ```rawdata``` and instead generate a dataframe with some random data

In [4]:
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

In [5]:
def wrangle_data(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    datalist=jsondata['data']
    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    index=pd.date_range(startdate, enddate, freq='7D')
    df=pd.DataFrame(index=index, columns=['occipiedBeds', 'hospital'])

    for entry in datalist: # each entry is a dictionary with date, hospital and occipiedBeds
        date=parse_date(entry['date'])
        for column in ['occipiedBeds', 'hospital']:
            # check that nothing is there yet - just in case some dates are duplicated,
            # maybe with data for different columns in each entry
            if pd.isna(df.loc[date, column]): 
                # replace None with 0 in our data 
                value=entry[column] if entry[column]!=None else 0
                df.loc[date, column]=value
    df.fillna(0, inplace=True)
    return df

# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in this cell as below:
df=wrangle_data(jsondata) # df is the dataframe for plotting
df

,occipiedBeds,hospital
2020-03-20,0,1277
2020-03-27,0,9633
2020-04-03,1788,17918
2020-04-10,2820,17763
2020-04-17,2780,12576
...,...,...
2023-10-27,0,3062
2023-11-03,0,2506
2023-11-10,0,2300
2023-11-17,0,2012


## Download current data

Give your users an option to refresh the dataset - a "refresh" button will do. The button callback should
* call the code that accesses the API and download some fresh raw data;
* wrangle that data into a dataframe and update the corresponding (global) variable for plotting (here, ```df```);
* optionally: force a redraw of the graph and give the user some fredback.

Once you get it to work, you may want to wrap your API call inside an exception handler, so that the user is informed, the "canned" data are not overwritten and nothing crashes if for any reason the server cannot be reached or data are not available.

After you refresh the data, graphs will not update until the user interacts with a widget. You can trick ```iPywidgets``` into redrawing the graph by simulating interaction, as in the ```refresh_graph``` function we define in the Graph and Analysis section below.

In this example, clicking on the button below just generates some more random data and refreshes the graph. The button should read *Fetch Data*. If you see anything else, take a deep breath :)

In [26]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_api():
    """ Accesses the PHE API. Return data as a like-for-like replacement for the "canned" data loaded from the JSON file. """
    filters = ['areaType=nation', 'areaName=England']
    structure = {
        "date": "date",
        "occipiedBeds": "covidOccupiedMVBedsWeekly",
        "hospital": "newAdmissionsWeekly",
    }
    
    # handle error
    try:
        occipiedBeds = Cov19API(filters=filters, structure=structure).get_json()
        with open("occipiedBeds.json", "wt") as OUTF:
            json.dump(occipiedBeds, OUTF)
        with open("occipiedBeds.json", "rt") as INFILE:
            jsondata=json.load(INFILE)
        return jsondata # return data read from the API
    except Exception as ex:
        print(f'Exception [{ex}]')
        return None

In [27]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.
    apidata=access_api()
    if apidata:
        # wrangle the data and overwrite the dataframe for plotting
        global df
        df=wrangle_data(apidata)
        # the graph won't refresh until the user interacts with the widget.
        # this function simulates the interaction, see Graph and Analysis below.
        # The function needs to be adapted to your graph; you can omit this call
        # in the first instance
        refresh_graph()
        # after all is done, you can switch the icon on the button to a "check" sign
        # and optionally disable the button - it won't be needed again. If you are 
        # implementing error handling, you can use icons "unlink" or "times" and 
        # change the button text to "Unavailable" when the api call fails.
        apibutton.icon="check"
        apibutton.description="Updated"
        apibutton.button_style="success"
    else:
        apibutton.icon="unlink"
        apibutton.description="Unavailable"
        apibutton.button_style="danger"

    
apibutton=wdg.Button(
    description='Refresh', # you may want to change this...
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Click to download current hospital admissions and ventilator bed occupancy of Public Health England data",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='download'
)

# remember to register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets

display(apibutton)

# run all cells before clicking on this button

Button(button_style='info', description='Refresh', icon='download', style=ButtonStyle(), tooltip='Click to dow…

## Graphs and Analysis

Include at least one graph with interactive controls, as well as some instructions for the user and/or comments on what the graph represents and how it should be explored (this example shows two random walks)

In [8]:
def plot_type(type):
    """ Our sample graph plotting function """
    df[list(type)].plot()
    plt.show() # important! update won't work properly without this

# a sample widget
typecols=wdg.SelectMultiple(
    options=['hospital', 'occipiedBeds'],
    value=['hospital', 'occipiedBeds'],
    rows=2,
    description='Type: ',
    disabled=False,
)

def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. This is a bit of a gimmick; it
    needs to be customised for one of your widgets. """
    current=typecols.value
    if current[0]==typecols.options[0]:
        other=typecols.options[1]
    else:
        other=typecols.options[0]
    typecols.value=[other] # forces the redraw
    typecols.value=current # now we can change it back

# connect the plotting function and the widget    
graph=wdg.interactive_output(plot_type, {'type': typecols})

# actually displays the graph
display(typecols, graph)

SelectMultiple(description='Type: ', index=(0, 1), options=('hospital', 'occipiedBeds'), rows=2, value=('hospi…

Output()

## Deploying the dashboard

Once your code is ready and you are satisfied with the appearance of the graphs, replace all the text boxes above with the explanations you would like a dashboard user to see. The next step is deploying the dashboard online - there are several [options](https://voila.readthedocs.io/en/stable/deploy.html) for this, we suggest deploying as a [Binder](https://mybinder.org/). This is basically the same technique that has been used to package this tutorial and to deploy this template dashboard. The instructions may seem a bit involved, but the actual steps are surprisingly easy - we will be going through them together during a live session. You will need an account on [GitHub](https://github.com/) for this - if you don't have one already, now it's the time to create it. 

**Author and Copyright Notice** Remember that if you deploy this dashboard as a Binder it will be publicly accessible. Take credit for your work! Also acknowledge your sources: Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england) and on the [DIY Covid Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash), Copyright (C) Fabrizio Smeraldi 2020,2023. Released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).